## Connect to Drive and importing dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
pip install swifter

In [3]:
import pandas as pd
import numpy as np
import swifter
import re
from sklearn.model_selection import KFold
import statistics
import warnings
import seaborn as sns
from tqdm import tqdm
warnings.filterwarnings("ignore")

pd.options.mode.chained_assignment = None

## Read the data

In [4]:
pathTrain = "/content/drive/MyDrive/NLP Study Group/Problem 4/train.dep.txt"
pathTest = "/content/drive/MyDrive/NLP Study Group/Problem 4/test.dep.txt"

In [5]:
columns = ['index','word', 'pos', 'pos1', 'unused', 'parent', 'type']

In [6]:
train = pd.read_csv(pathTrain, sep = "\t", names = columns)
train = train[['word', 'pos', 'parent', 'type']]
train.reset_index(inplace = True)
train

,index,word,pos,parent,type
0,1,in,IN,43,PP
1,2,an,DT,5,DEP
2,3,oct.,NNP,5,DEP
3,4,19,CD,5,DEP
4,5,review,NN,1,NP
...,...,...,...,...,...
5275,4,",",",",7,DEP
5276,5,'','',7,DEP
5277,6,he,PRP,7,NP-SBJ
5278,7,says,VBZ,0,ROOT


In [7]:
test = pd.read_csv(pathTest, sep = "\t", names = columns)
test = test[['word', 'pos', 'parent', 'type']]
test.reset_index(inplace = True)
test

,index,word,pos,parent,type
0,1,no,UH,4,INTJ
1,2,",",",",4,DEP
2,3,it,PRP,4,NP-SBJ
3,4,was,VBD,0,ROOT
4,5,n't,RB,4,DEP
...,...,...,...,...,...
4634,19,by,IN,6,PP
4635,20,year,NN,21,DEP
4636,21,'s,POS,22,NP
4637,22,end,NN,19,NP


## Class and Functions

In [8]:
def checkVertex(a, b, df):
  temp = df[df["wordKey"] == a]

  flag = len(temp[temp["parentKey"] == b])
  #display(temp[temp["parent"] == b])

  if flag is 0: 
    return False 
  else:
    return True

In [9]:
def getPOS(word, df):
  if word is not "ROOT":
    return df["pos"][df['word'] == word].values[0]
  else:
    return "NONE"

In [10]:
def getValue(key, df):
  if key is 'ROOT':
    return 'ROOT'
  else:
    return df['word'][df['wordKey'] == key].values[0] 

In [11]:
def dataFramePerSentence(dframe, numbSentence):
  tempList = []
  startPoint = 0

  for i in range(numbSentence):
    temp = dframe[startPoint:].copy()  
    temp.reset_index(inplace = True)
    temp['level_0'] = temp['level_0'] - temp['level_0'][0] + 1
    temp = temp[temp['level_0'] == temp['index']]
    startPoint = startPoint + len(temp)

    temp.drop(['index','level_0'], axis=1, inplace = True)

    temp.parent = temp.parent - 1
    temp['parentKey'] = temp.parent

    for index, row in temp.iterrows():
      if row.parent is -1: 
        temp['parent'][index] = 'ROOT'
      elif not isinstance(temp['parent'][index], str):
        key = temp['parent'][index]
        temp['parent'][index] = temp['word'][key]

    temp['wordKey'] = temp.index
    tempList.append(temp) 

  return tempList

In [12]:
def generateFeatures(dfList, printStep = False):
  featuresHolder = []

  for df in dfList:
    buffer = list(df['wordKey'])
    stack = ['ROOT']

    while not(len(buffer) is 0 and len(stack) is 2):   
      if len(stack) > 2:
        if checkVertex(stack[-2], stack[-1], df):  #left
            if printStep: print("⦿LEFT: ",stack[-2]," <-- ", stack[-1])
            flag = "left"
            stack.pop(-2)
        elif checkVertex(stack[-1], stack[-2], df):  #right
          countRemainingDependant = 0

          for remainingWord in buffer:
            if checkVertex(remainingWord, stack[-1], df) == True:
              countRemainingDependant += 1
            if checkVertex(stack[-1], remainingWord, df) == True:
              countRemainingDependant += 1         

          if countRemainingDependant is not 0:
            stack.append(buffer.pop(0))
            if printStep: print("⦿shift right")
          else:
            if printStep: print("⦿RIGHT: ",stack[-2]," --> ", stack[-1])    
            flag = "right"
            stack.pop(-1)

        else:
          if printStep: print("⦿shift inner")
          flag = "shift"
          stack.append(buffer.pop(0))
      else:
        if printStep: print("⦿shift outer")
        flag = "shift"
        stack.append(buffer.pop(0))
                  
      if printStep: print("stack: ", stack)
      if printStep: print("buffer: ", buffer)
      if printStep: print("==================")


      stack1 = getValue(stack[-1], df)
      if len(stack) > 1:
        stack2 = getValue(stack[-2], df)
        if len(buffer):
          buffer1 = getValue(buffer[0], df)
          featuresHolder.append([stack2, stack1, buffer1,
                          getPOS(stack2, df), getPOS(stack1, df), getPOS(buffer1, df),
                          len(stack), len(buffer), flag])
        else:
          featuresHolder.append([stack2, stack1, 'EMPTY-BUFFER', 
                          getPOS(stack2, df), getPOS(stack1, df), 'EMPTY-BUFFER',
                          len(stack), len(buffer), flag])
      else:
        if len(buffer):
          buffer1 = getValue(buffer[0], df)
          featuresHolder.append(['ROOT', stack1, buffer1, 
                          getPOS('ROOT', df), getPOS(stack1, df), getPOS(buffer1, df),
                          len(stack), len(buffer), flag])
        else:
          featuresHolder.append(['ROOT', stack1, 'EMPTY-BUFFER', 
                          getPOS('ROOT', df), getPOS(stack1, df), 'EMPTY-BUFFER',
                          len(stack), len(buffer), flag])

  return pd.DataFrame(featuresHolder, columns =['stack0', 'stack1', 'buffer0', 'posstack0', 'posstack1', 'posbuffer0', 'lenstack', 'lenbuffer', 'target'])

In [181]:
class Perceptron:
  def __init__(self):
    self._weights = None
    self._learningRate = 0.01
    self._epoch = 10
    self._numFeatures = None
    self._numClass = None
    self._x = None
    self._y = None

  def fit(self, x, y, lr, epoch):
    self._x = np.array(x)
    self._y = np.array(y)
    self._numFeatures = len(x[0])
    self._numClass = len(y[0])
    self._learningRate = lr
    self._epoch = epoch

    self._weights = np.random.rand(self._numClass, self._numFeatures)

    for ep in range(self._epoch):
      listResult = []

      for i in range(len(self._x)):
        linearOutput = self.linearFunction(self._x[i], self._weights)
        activatedOutput = self.activationFunction(linearOutput)
        softmaxOutput = self.softmaxFunction(activatedOutput)
        listResult.append(softmaxOutput)
        self._weights = self.updateWeights(x[i], activatedOutput, y[i], softmaxOutput)

      print("epoch: ", ep, 
        "\tloss: ", self.crossEntropyLoss(np.asarray(listResult), self._y), 
        "\taccuracy: ", self.accuracy(np.asarray(listResult), y)
      )

  def linearFunction(self, inputFeatures, weights):
    return [np.dot(inputFeatures, weight) for weight in weights]
  
  def activationFunction(self, linearResults):
    return [1 / ( 1 + np.exp(-(linearResult))) for linearResult in linearResults]

  def softmaxFunction(self, activatedResults):
    denominator = sum(self.activationFunction(activatedResults))
    return [self.activationFunction([output])[0] / denominator for output in activatedResults]

  def crossEntropyLoss(self, yPredicted, yActual):
    cummulativeLoss = 0
    for i in range(len(yPredicted)):
      loss = -1 * np.dot(yActual[i], np.log(yPredicted[i]))
      cummulativeLoss = cummulativeLoss + loss

    return loss / len(yPredicted)

  def accuracy(self, yPredicted, yActual):
    ypred = np.argmax(yPredicted, axis = 0)
    yact = np.argmax(yActual, axis = 0)
    correct = (yact == ypred)
    return correct.sum() / correct.size


  def updateWeights(self, x, activated, yactual, ypred):
    temp = np.zeros((self._numClass, self._numFeatures))
    
    for i in range(len(temp)): #output
      for j in range(len(temp[0])): #input
        d = (ypred[i] - yactual[i]) * activated[i] * (1 - activated[i]) * x[j]
        temp[i][j] = self._weights[i][j] - (self._learningRate * d)

    return temp

  def predict(self, x):
    x = np.array(x)
    listResult = []

    for i in range(len(x)):
      linearOutput = self.linearFunction(x[i], self._weights)
      activatedOutput = self.activationFunction(linearOutput)
      softmaxOutput = self.softmaxFunction(activatedOutput)
      listResult.append(softmaxOutput)

    return listResult


In [182]:
a = [[1,1,1], [0,0,0], [2,2,2]]
b = [[0,1,0], [1,0,0], [0,1,0]]

pp = Perceptron()
pp.fit(a, b, 0.1, 20)
np.argmax(b, axis=0) 

epoch:  0 	loss:  0.36232119014173253 	accuracy:  0.6666666666666666
epoch:  1 	loss:  0.36191708838759223 	accuracy:  0.6666666666666666
epoch:  2 	loss:  0.3614872871156856 	accuracy:  0.6666666666666666
epoch:  3 	loss:  0.36102696043884713 	accuracy:  0.6666666666666666
epoch:  4 	loss:  0.3605307749451152 	accuracy:  0.6666666666666666
epoch:  5 	loss:  0.35999277991364204 	accuracy:  0.6666666666666666
epoch:  6 	loss:  0.3594062908855933 	accuracy:  0.6666666666666666
epoch:  7 	loss:  0.35876376813332694 	accuracy:  0.6666666666666666
epoch:  8 	loss:  0.35805669423537173 	accuracy:  0.6666666666666666
epoch:  9 	loss:  0.3572754590083546 	accuracy:  0.6666666666666666
epoch:  10 	loss:  0.35640926603793605 	accuracy:  0.6666666666666666
epoch:  11 	loss:  0.3554460836114705 	accuracy:  0.6666666666666666
epoch:  12 	loss:  0.35437267448830356 	accuracy:  0.6666666666666666
epoch:  13 	loss:  0.35317475363785283 	accuracy:  0.6666666666666666
epoch:  14 	loss:  0.35183733955724

array([1, 0, 0])

In [14]:
class ShiftReduce:
  def __init__(self):
    self._sentence = None
    self._stack = None

  def process(self, sentence, model):
    self.sentence = sentence
    self._stack = ['ROOT']

    while len(self._sentence) > 0 and len(self._stack) > 1:
      # if model.predict() == 'shift'
      #   shift()
      # elif model.predict() == 'left'
      #   left()
      # elif model.predict() == 'right'
      #   right()

  #def shift(self)
  #def left(self)
  #def right(self)

SyntaxError: ignored

## Get dataframe for every sentence

In [15]:
dfTrainList = dataFramePerSentence(train, 200)
dfTestList = dataFramePerSentence(test, 200)

In [16]:
dfTrainList[1]

,word,pos,parent,type,parentKey,wordKey
0,ms.,NNP,haag,DEP,1,0
1,haag,NNP,plays,NP-SBJ,2,1
2,plays,VBZ,ROOT,ROOT,-1,2
3,elianti,NNP,plays,NP-OBJ,2,3
4,.,.,plays,DEP,2,4


In [17]:
dfTestList[0]

,word,pos,parent,type,parentKey,wordKey
0,no,UH,was,INTJ,3,0
1,",",",",was,DEP,3,1
2,it,PRP,was,NP-SBJ,3,2
3,was,VBD,ROOT,ROOT,-1,3
4,n't,RB,was,DEP,3,4
5,black,JJ,monday,DEP,6,5
6,monday,NNP,was,NP-PRD,3,6
7,.,.,was,DEP,3,7


## Check generateFeatures function

In [18]:
feat = generateFeatures(dfTestList[170:171], True)
feat

⦿shift outer
stack:  ['ROOT', 0]
buffer:  [1, 2, 3, 4, 5, 6, 7, 8]
⦿shift outer
stack:  ['ROOT', 0, 1]
buffer:  [2, 3, 4, 5, 6, 7, 8]
⦿LEFT:  0  <--  1
stack:  ['ROOT', 1]
buffer:  [2, 3, 4, 5, 6, 7, 8]
⦿shift outer
stack:  ['ROOT', 1, 2]
buffer:  [3, 4, 5, 6, 7, 8]
⦿shift inner
stack:  ['ROOT', 1, 2, 3]
buffer:  [4, 5, 6, 7, 8]
⦿LEFT:  2  <--  3
stack:  ['ROOT', 1, 3]
buffer:  [4, 5, 6, 7, 8]
⦿LEFT:  1  <--  3
stack:  ['ROOT', 3]
buffer:  [4, 5, 6, 7, 8]
⦿shift outer
stack:  ['ROOT', 3, 4]
buffer:  [5, 6, 7, 8]
⦿shift right
stack:  ['ROOT', 3, 4, 5]
buffer:  [6, 7, 8]
⦿shift right
stack:  ['ROOT', 3, 4, 5, 6]
buffer:  [7, 8]
⦿shift inner
stack:  ['ROOT', 3, 4, 5, 6, 7]
buffer:  [8]
⦿LEFT:  6  <--  7
stack:  ['ROOT', 3, 4, 5, 7]
buffer:  [8]
⦿RIGHT:  5  -->  7
stack:  ['ROOT', 3, 4, 5]
buffer:  [8]
⦿RIGHT:  4  -->  5
stack:  ['ROOT', 3, 4]
buffer:  [8]
⦿RIGHT:  3  -->  4
stack:  ['ROOT', 3]
buffer:  [8]
⦿shift outer
stack:  ['ROOT', 3, 8]
buffer:  []
⦿RIGHT:  3  -->  8
stack:  ['ROOT',

,stack0,stack1,buffer0,posstack0,posstack1,posbuffer0,lenstack,lenbuffer,target
0,ROOT,the,centers,NONE,DT,NNS,2,8,shift
1,the,centers,normally,DT,NNS,RB,3,7,shift
2,ROOT,centers,normally,NONE,NNS,RB,2,7,left
3,centers,normally,are,NNS,RB,VBP,3,6,shift
4,normally,are,closed,RB,VBP,VBN,4,5,shift
5,centers,are,closed,NNS,VBP,VBN,3,5,left
6,ROOT,are,closed,NONE,VBP,VBN,2,5,left
7,are,closed,through,VBP,VBN,IN,3,4,shift
8,closed,through,the,VBN,IN,DT,4,3,shift
9,through,the,weekend,IN,DT,NN,5,2,shift


## Generate features for Training and Testing

In [19]:
trainFeatures = generateFeatures(dfTrainList, False)
testFeatures = generateFeatures(dfTestList, False)

In [20]:
trainFeatures

,stack0,stack1,buffer0,posstack0,posstack1,posbuffer0,lenstack,lenbuffer,target
0,ROOT,in,an,NONE,IN,DT,2,48,shift
1,in,an,oct.,IN,DT,NNP,3,47,shift
2,an,oct.,19,DT,NNP,CD,4,46,shift
3,oct.,19,review,NNP,CD,NN,5,45,shift
4,19,review,of,CD,NN,IN,6,44,shift
...,...,...,...,...,...,...,...,...,...
10355,kiddies,says,.,NNS,VBZ,.,4,1,left
10356,``,says,.,``,VBZ,.,3,1,left
10357,ROOT,says,.,NONE,VBZ,.,2,1,left
10358,says,.,EMPTY-BUFFER,VBZ,.,EMPTY-BUFFER,3,0,shift


In [21]:
testFeatures

,stack0,stack1,buffer0,posstack0,posstack1,posbuffer0,lenstack,lenbuffer,target
0,ROOT,no,",",NONE,UH,",",2,7,shift
1,no,",",it,UH,",",PRP,3,6,shift
2,",",it,was,",",PRP,VBD,4,5,shift
3,it,was,n't,PRP,VBD,RB,5,4,shift
4,",",was,n't,",",VBD,RB,4,4,left
...,...,...,...,...,...,...,...,...,...
9073,supposed,send,.,VBN,VB,.,4,1,right
9074,is,supposed,.,VBZ,VBN,.,3,1,right
9075,ROOT,is,.,NONE,VBZ,.,2,1,right
9076,is,.,EMPTY-BUFFER,VBZ,.,EMPTY-BUFFER,3,0,shift
